In [1]:
%%capture
#%pip install protobuf==3.20.1
%pip install transformers[torch]
%pip install -q sentencepiece
%pip install datasets==2.13.1
%pip install evaluate
%pip install rouge_score

In [2]:
QPATH = "Quantlet/5-domain-pre-training"

In [3]:
import sys

sys.path.append("../4-qode2desc")
IN_COLAB = "google.colab" in sys.modules

import os

if IN_COLAB:
    os.chdir(
        f"/content/drive/MyDrive/ColabNotebooks/IRTG/Encode_the_Qode/Encode-the-Qode/{QPATH}"
    )
else:
    %load_ext lab_black

In [4]:
import pickle
import json
import re
import sys
from IPython.display import display
import datetime

from tqdm import tqdm
import pandas as pd
import numpy as np

tqdm.pandas()


import matplotlib.pyplot as plt
import seaborn as sns

import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelWithLMHead, SummarizationPipeline
from transformers import AdamW
from datasets import load_dataset

from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
)

os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["WANDB_DISABLED"] = "true"

import nltk

nltk.download("punkt")
import evaluate

from sklearn.model_selection import train_test_split

[nltk_data] Downloading package punkt to
[nltk_data]     /home/RDC/zinovyee.hub/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
import importlib
import analysis_modules

importlib.reload(analysis_modules)

[nltk_data] Downloading package punkt to
[nltk_data]     /home/RDC/zinovyee.hub/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/RDC/zinovyee.hub/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


<module 'analysis_modules' from '/usr/net/zinovyee.hub/IRTG/MLSC/Encode-the-Qode/Quantlet/5-domain-pre-training/../4-qode2desc/analysis_modules.py'>

In [6]:
model_name = "CodeT5"

DATE = "202310145"

SAMPLE = "test"
if SAMPLE == "test":
    load_best_model_at_end = False
else:
    load_best_model_at_end = None

# tokenization
encoder_max_length = 300
decoder_max_length = 30
RS = 1
LR = 5e-5

EPOCHS = 4
TRAIN_BATCH = 4
EVAL_BATCH = 4

WARMUP_STEPS = 500
WEIGHT_DECAY = 0.1
LOGGING_STEPS = 100
SAVE_TOTAL_LIM = 4
SAVE_STRATEGY = "steps"

LABEL_SMOOTHING = 0.1
PREDICT_GENERATE = True

EVAL_COLUMNS = [
    "eval_loss",
    "eval_rouge1",
    "eval_rouge2",
    "eval_rougeL",
    "eval_rougeLsum",
    "eval_bleu",
    "eval_gen_len",
]


analysis_name = (
    model_name
    + "-"
    + str(EPOCHS)
    + "-"
    + str(encoder_max_length)
    + "-"
    + str(decoder_max_length)
    + "-"
    + str(TRAIN_BATCH)
)

analysis_name = analysis_name + "-" + DATE + "_" + str(RS)

print(analysis_name)

CodeT5-4-300-15-4-202310145_1


In [7]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()

In [8]:
print(analysis_name)
analysis_modules.scs_analyze(
    analysis_name=analysis_name,
    model_name=model_name,
    train_data_path=f"../../data/preprocessed/ArXiv/",
    train_data_name=f"arxiv_pretrain_train.json",
    val_data_path=f"../../data/preprocessed/ArXiv/",
    val_data_name=f"arxiv_pretrain_test.json",
    encoder_max_length=encoder_max_length,
    decoder_max_length=decoder_max_length,
    random_state=RS,
    eval_columns_list=EVAL_COLUMNS,
    learning_rate=LR,
    epochs=EPOCHS,
    train_batch=TRAIN_BATCH,
    eval_batch=EVAL_BATCH,
    warmup_steps=WARMUP_STEPS,
    weight_decay=WEIGHT_DECAY,
    logging_stes=LOGGING_STEPS,
    save_total_lim=SAVE_TOTAL_LIM,
    save_strategy=SAVE_STRATEGY,
    label_smooting=LABEL_SMOOTHING,
    predict_generate=PREDICT_GENERATE,
    load_best_model_at_end=load_best_model_at_end,
)

CodeT5-4-300-15-4-202310145_1
CodeT5-4-300-15-4-202310145_1
cuda


/home/RDC/zinovyee.hub/.local/lib/python3.9/site-packages/transformers/models/auto/modeling_auto.py:1322: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


cuda


Found cached dataset json (/home/RDC/zinovyee.hub/.cache/huggingface/datasets/json/default-a416704e79b803c8/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)


  0%|          | 0/1 [00:00<?, ?it/s]

...

Step,Training Loss
100,5.304500
200,4.350000
300,4.124100
400,3.826600
500,3.798500
600,3.613500
700,3.610500
800,3.522400
900,3.507200
1000,3.456200


/home/RDC/zinovyee.hub/.conda/envs/encode_code/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


   eval_loss  eval_rouge1  eval_rouge2  eval_rougeL  eval_rougeLsum  \
0      2.697        0.483        0.305         0.45            0.45   

   eval_bleu  eval_gen_len  
0      0.192        13.868  


/home/RDC/zinovyee.hub/.local/lib/python3.9/site-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


__________
Original: Comparing Different Deep Learning Architectures for Classification of Chest Radiographs
Summary before Tuning: This function is used to classify chest radiographs.
Summary after Tuning: Deep Learning on Chest Radiographs: A Compar
__________


__________
Original: TGHop: An Explainable, Efficient and Lightweight Method for Texture Generation
Summary before Tuning: This method is a lightweight method that is used to generate a high quality texture for a
Summary after Tuning: TGHop: An Explainable Texture Generation PixelHop++
__________


__________
Original: Unidentified Floating Object detection in maritime environment using dictionary learning
Summary before Tuning: Maritime domain is one of the most challenging scenarios for the Mar
Summary after Tuning: Unidentified Floating Objects in Maritime using K
__________


__________
Original: Understanding Graph Neural Networks from Graph Signal Denoising Perspectives
Summary before Tuning: Graph neural networks have 

In [9]:
gc.collect()
torch.cuda.empty_cache()